# Monte-Carlo Off-Police

### Instalação de pacotes

In [2]:
from IPython.display import clear_output
!apt-get install ffmpeg freeglut3-dev xvfb  
!pip install gym[all]==00.25.1
!pip install gym[atari,accept-rom-license]==00.25.1
!pip install pyglet
!pip install stable-baselines3[extra]
!pip install optuna
clear_output()

'apt-get' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


^C


In [ ]:
!mkdir log_project

### Imports


In [3]:
import gym
import numpy as np
import tensorboard
import matplotlib.pyplot as plt
import time

%load_ext tensorboard

import gym
import numpy as np


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Para salvar vídeo

In [ ]:
# Set up fake display; otherwise rendering will fail
import os
!pip install stable-baselines3[extra]
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

A gravação é feita com o wrapper [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder).

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record the given number of steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [ ]:
# ideias adaptadas de : https://www.anyscale.com/blog/an-introduction-to-reinforcement-learning-with-openai-gym-rllib-and-google
from base64 import b64encode
from IPython.display import HTML
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  html_code = f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'
  return HTML(html_code)

# Código

In [4]:
ENV_NAME = "Taxi-v3"  
 
env = gym.make(ENV_NAME)

## Off-policy

Off-Policy Padrão

In [8]:
def choose_action(Q, state):
    return np.argmax(Q[state])

def choose_actionB(num_actions):
    return np.random.randint(0, num_actions)

# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarloOffP(env, episodes, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)
    
    num_actions = env.action_space.n
    
    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))
    C = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []
        
        state = env.reset()
    
        # PARTE 1: executa um episódio completo
        while done != True:   
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios 
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()
                
            # escolhe a próxima ação -- usa epsilon-greedy
            action = choose_actionB(num_actions)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )
            
            sum_rewards += reward
            state = next_state
        
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso 
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # PARTE 2: atualiza Q (e a política, implicitamente)
        Gt = 0
        W = 1
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt
            C[s,a] = C[s,a] + W
            delta = W * (Gt - Q[s,a])
            Q[s,a] = Q[s,a] + (1/C[s,a])* delta
            best = choose_action(Q,s)
            if best != a: break
            W = W*(1/(1/num_actions))

    return sum_rewards_per_ep, Q


Off-policy com weighted importance sampling

In [ ]:
# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarloOffP_weighted(env, episodes, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)
    
    num_actions = env.action_space.n
    
    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))
    C = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []
        
        state = env.reset()
    
        # PARTE 1: executa um episódio completo
        while done != True:   
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios 
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()
                
            # escolhe a próxima ação -- usa epsilon-greedy
            action = choose_actionB(num_actions)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )
            
            sum_rewards += reward
            state = next_state
        
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso 
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # PARTE 2: atualiza Q (e a política, implicitamente)
        Gt = 0
        W = 1
        gamma_fact = 1
        for (i, (s, a, r)) in enumerate(reversed(ep_trajectory)):
            Gt = Gt + r
            if i == 1 and gamma != 1:
                gamma_fact *= (1-gamma)/gamma
            else:
                gamma_fact *= gamma
            actual_w = W * gamma_fact
            C[s,a] = C[s,a] + actual_w
            delta = actual_w * (Gt - Q[s,a])
            Q[s,a] = Q[s,a] + (1/C[s,a])* delta
            best = choose_action(Q,s)
            W = W*(1/(1/num_actions))
            if best != a: break

    return sum_rewards_per_ep, Q

## On-Policy

In [12]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def pi_policy(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])


# Algoritmo Monte-Carlo de Controle, variante "toda-visita".
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_montecarloOnP(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)
    
    num_actions = env.action_space.n
    
    # inicializa a tabela Q toda com zero,
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-discontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        ep_trajectory = []
        
        state = env.reset()
    
        # PARTE 1: executa um episódio completo
        while done != True:   
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios 
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()
                
            # escolhe a próxima ação -- usa epsilon-greedy
            action = pi_policy(Q, state, num_actions, epsilon)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            # adiciona a tripla que representa este passo
            ep_trajectory.append( (state, action, reward) )
            
            sum_rewards += reward
            state = next_state
        
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso 
        if (i+1) % 100 == 0:
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

        # PARTE 2: atualiza Q (e a política, implicitamente)
        Gt = 0
        for (s, a, r) in reversed(ep_trajectory):
            Gt = r + gamma*Gt
            delta = Gt - Q[s,a]
            Q[s,a] = Q[s,a] + lr * delta

    return sum_rewards_per_ep, Q

### Execução Off-Policy Padrão

#### Otimiza Parâmetros

In [10]:
from numpy.random.mtrand import gamma

import optuna



ENV = gym.make("Taxi-v3")


# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_values(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    #bins1 = trial.suggest_int('bins1', 5, 100)
    #bins2 = trial.suggest_int('bins2', 5, 100)
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}")

    # roda o algoritmo e recebe os retornos não-descontados
    #env_wrapper = DiscreteObservationWrapper(ENV, [bins1,bins2])
    (returns, _) = run_montecarloOffP(env, 20000, gamma, eps, render=False)
    return sum(returns[-100:])/100 

d:\Arquivos de Programas\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new2_MC_offpolice', 
                            load_if_exists=True)
study.optimize(train_values, n_trials=20) 

#### Execução

In [6]:

env = gym.make("Taxi-v3")

In [ ]:
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.01
    GAMMA = 0.830525147061507
    EPSILON = 0.05919712699520377

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOffP(env, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOffP-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'offPolicyD')

    # test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

### Execução Off-Policy(Weighted samples)

#### Otimiza parâmetros

In [ ]:
from numpy.random.mtrand import gamma

import optuna



ENV = gym.make("Taxi-v3")


# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_values(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    #bins1 = trial.suggest_int('bins1', 5, 100)
    #bins2 = trial.suggest_int('bins2', 5, 100)
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}")

    # roda o algoritmo e recebe os retornos não-descontados
    #env_wrapper = DiscreteObservationWrapper(ENV, [bins1,bins2])
    (returns, _) = run_montecarloOffP_weighted(ENV, 20000, gamma, eps, render=False)
    return sum(returns[-100:])/100 

In [ ]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice_weighted', 
                            load_if_exists=True)
study.optimize(train_values, n_trials=20)

#### Execução

In [ ]:
env = gym.make("Taxi-v3")

In [ ]:
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.01
    GAMMA = 0.5723036890441425
    EPSILON = 0.1915757619563071

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOffP_weighted(env, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOffP_weighted-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'offPolicyD_weighted')

    # test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

### Execução On-Policy

#### Otimiza Parâmetros

In [ ]:
from numpy.random.mtrand import gamma

import optuna



ENV = gym.make("Taxi-v3")


# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_values(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    lr = trial.suggest_uniform('learning_rate', 0.001, 1.0)
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
   
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}")

    # roda o algoritmo e recebe os retornos não-descontados
    
    (returns, _) = run_montecarloOnP(ENV, 20000, lr, gamma, eps, render=False)
    return sum(returns[-100:])/100 

In [ ]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_onpolice', 
                            load_if_exists=True)
study.optimize(train_values, n_trials=20) 

#### Execução

In [13]:
ENV_NAME = "Taxi-v3"  
#ENV_NAME = "MountainCarContinuous-v0"  
#ENV_NAME = "LunarLander-v2"  
env = gym.make(ENV_NAME)

#parameters: {'learning_rate': 0.007617314337157292, 'gamma': 0.8762195844017038, 'epsilon': 0.1594319424288697}. Best is trial 16 with value: -2.2.
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.007617314337157292
    GAMMA = 0.8762195844017038
    EPSILON = 0.1594319424288697

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOnP(env, EPISODES, LR, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOnP-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'onPolicyD')

    # test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

Episode 100 Average Reward (last 100): -466.380
Episode 200 Average Reward (last 100): -907.180
Episode 300 Average Reward (last 100): -919.710
Episode 400 Average Reward (last 100): -845.390
Episode 500 Average Reward (last 100): -647.660
Episode 600 Average Reward (last 100): -461.720
Episode 700 Average Reward (last 100): -474.840
Episode 800 Average Reward (last 100): -393.980
Episode 900 Average Reward (last 100): -388.660
Episode 1000 Average Reward (last 100): -319.070
Episode 1100 Average Reward (last 100): -353.950
Episode 1200 Average Reward (last 100): -324.430
Episode 1300 Average Reward (last 100): -300.000
Episode 1400 Average Reward (last 100): -284.720
Episode 1500 Average Reward (last 100): -266.080
Episode 1600 Average Reward (last 100): -252.130
Episode 1700 Average Reward (last 100): -249.180
Episode 1800 Average Reward (last 100): -228.040
Episode 1900 Average Reward (last 100): -204.970
Episode 2000 Average Reward (last 100): -182.050
Episode 2100 Average Reward (

NameError: name 'plot_result' is not defined

## Contínuo

In [15]:

ENV_NAME = "MountainCar-v0"  

env = gym.make(ENV_NAME)

In [16]:

class GeneralDiscretizer:
    def __init__(self, env, bins_per_dimension):
        self.bins_per_dim = bins_per_dimension.copy()
        self.intervals_per_dim = []
        self.total_bins = 1
        for i, bins in enumerate(bins_per_dimension):
            self.intervals_per_dim.append(
                np.linspace(env.observation_space.low[i], env.observation_space.high[i], bins+1) )
            self.total_bins *= bins

    def to_single_bin(self, state):
        bin_vector = [(np.digitize(x=state[i], bins=intervals) - 1)
                      for i, intervals in enumerate(self.intervals_per_dim)]
        # print(bin_vector)
        return self._bin_vector_to_single_bin(bin_vector, len(bin_vector)-1)

    def _bin_vector_to_single_bin(self, vector, index):
        if index < 0:
            return 0
        return vector[index] + self.bins_per_dim[index] * self._bin_vector_to_single_bin(vector, index-1)

    def get_total_bins(self):
        return self.total_bins


class DiscreteObservationWrapper(gym.ObservationWrapper):
    '''Classe para converter espaços contínuos em espaços discretos.
    Esta classe converte ambientes de observações (estados) contínuos em ambientes de estados
    discretos. Especificamente, ele converte representações dadas na forma de array de valores float
    em um único inteiro $\geq$ não-negativo (>=0).
    
    Precisa passar para o construtor uma lista que informa em quantos "bins" vai ser discretizada 
    cada dimensão (ou seja, cada valor float) do espaço de estados original.
    '''
    
    def __init__(self, env, BINS_PER_DIMENSION):
        super().__init__(env)
        # cria um GeneralDiscretizer para converter um array de valores float em um único inteiro >= 0
        # precisa dizer em quantos "bins" vai ser discretizada cada dimensão
        self.discretizer = GeneralDiscretizer(env, BINS_PER_DIMENSION)
        self.observation_space = gym.spaces.Discrete(self.discretizer.get_total_bins())

    def observation(self, obs):
        return self.discretizer.to_single_bin(obs)


In [ ]:
from numpy.random.mtrand import gamma

import optuna



# Esta função faz um treinamento com o Expected-SARSA, usando parâmetros sugeridos pelo Optuna.
# Retorna a média dos retornos dos últimos 100 episódios.
def train_valuesOf(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    bins1 = trial.suggest_int('bins1', 5.0, 80.0)
    bins2 = trial.suggest_int('bins2', 10.0, 90.0)
    
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}, bins1={bins1},bins2={bins2}")

    # roda o algoritmo e recebe os retornos não-descontados
    env_wrapper = DiscreteObservationWrapper(env, [bins1,bins2])
    (returns, _) = run_montecarloOffP(env_wrapper, 20000, gamma,eps, render=False)
    return sum(returns[-100:])/100

In [ ]:
def train_valuesOn(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    bins1 = trial.suggest_int('bins1', 5.0, 80.0)
    bins2 = trial.suggest_int('bins2', 10.0, 90.0)
   
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma}, bins1={bins1},bins2={bins2}")

    # roda o algoritmo e recebe os retornos não-descontados
    env_wrapper = DiscreteObservationWrapper(env, [bins1,bins2])
    (returns, _) = run_montecarloOnP(env_wrapper, 20000, gamma, eps, render=False)
    return sum(returns[-100:])/100

In [ ]:
study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice_cont_bins_on-Policy', 
                            load_if_exists=True)


study2 = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice_cont_bins_off-policy', 
                            load_if_exists=True)


study.optimize(train_valuesOn, n_trials=30) 


In [ ]:
study2.optimize(train_valuesOf, n_trials=30)

In [ ]:
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    GAMMA = 0.830525147061507
    EPSILON = 0.05919712699520377

    print("Ambiente Contínuo")
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    print("Monte-Carlo On-Policy")
    env_wrapper = DiscreteObservationWrapper(env, [70,70])
    rewardsOn, QtableOn = run_montecarloOnP(env_wrapper, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))
    filename = f"results/montecarloOnPolicyCont-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'onPolicyC')

    #test_greedy_Q_policy(env, QtableOn, 10, True)


    

In [ ]:
    env_wrapper = DiscreteObservationWrapper(env, [70,70])
    print("Monte-Carlo Off-Policy")
    env_wrapper = DiscreteObservationWrapper(env, [70,70])
    rewardsOff, QtableOff = run_montecarloOffP(env_wrapper, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOffPolicyCont-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'offPolicyC')

    #test_greedy_Q_policy(env, QtableOff, 10, True)
    env.close()

Continuo Ambiente 2 ()

In [ ]:
!pip install gym[box2d]

In [2]:
# ENV_NAME = "CarRacing-v1"  

# env = gym.make(ENV_NAME)
env = gym.make("Pendulum-v1")

# vamos ver como é um estado deste ambiente?
print("Espaço de estados/observações: ", env.observation_space)
print("  - formato: ", env.observation_space.shape)
print("  - exemplo: ", env.reset())

Espaço de estados/observações:  Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)
  - formato:  (3,)
  - exemplo:  [ 0.90833104 -0.41825196  0.8534491 ]


In [ ]:
def train_valuesOnCar(trial : optuna.Trial):
    
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    gamma = trial.suggest_uniform('gamma', 0.02, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    # bins1 = trial.suggest_int('bins1', -1, 1)
    bins2 = trial.suggest_int('bins2', 5.0, 30.0)
    bins3 = trial.suggest_int('bins3', 5.0, 10.0)
   
    
    print(f"\nTRIAL #{trial.number}: eps={eps}, gamma={gamma},bins2={bins2},bins3={bins3}")

    # roda o algoritmo e recebe os retornos não-descontados
    env_wrapper = DiscreteObservationWrapper(env, [bins2,bins3])
    (returns, _) = run_montecarloOffP(env_wrapper, 2000, gamma, eps, render=False)
    return sum(returns[-100:])/100

In [ ]:

study = optuna.create_study(direction='maximize', 
                            storage='sqlite:///optuna_studies.db', 
                            study_name= 'new_MC_offpolice_cont_bins_off-policy', 
                            load_if_exists=True)

study.optimize(train_valuesOnCar, n_trials=10) 

In [ ]:
if __name__ == "__main__":
    r_max_plot = 10

    EPISODES = 100000
    LR = 0.01
    GAMMA = 0.830525147061507
    EPSILON = 0.05919712699520377
 
    #env_wrapper = DiscreteObservationWrapper(env, [1,80,90])

    env = gym.make("FrozenLake-v1")

    
    # Roda o algoritmo Monte-Carlo para o problema de controle (ou seja, para achar a política ótima)
    rewards, Qtable = run_montecarloOffP(env, EPISODES, GAMMA, EPSILON, render=False)
    print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

    # Mostra um gráfico de episódios x retornos (não descontados)
    # Se quiser salvar, passe o nome do arquivo no 3o parâmetro
    filename = f"results/montecarloOffPCar-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
    plot_result(rewards, r_max_plot,100, 'offPolicyC-CarRacing')

    # test_greedy_Q_policy(env, Qtable, 10, True)
    env.close()

In [ ]:
rewards, Qtable = run_montecarloOnP(env, EPISODES, GAMMA, EPSILON, render=False)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

filename = f"results/montecarloOffPCar-{ENV_NAME.lower()[0:8]}-ep{EPISODES}.png"
plot_result(rewards, r_max_plot,100, 'onPolicyC-CarRacing')

   
env.close()

# Plot Result

In [ ]:
def test_greedy_Q_policy(env, Q, num_episodes=100, render=False, render_wait=0.01, video=None):
    """
    Avalia a política gulosa (greedy) definida implicitamente por uma Q-table.
    Ou seja, executa, em todo estado s, a ação "a = argmax Q(s,a)".
    - env: o ambiente
    - Q: a Q-table (tabela Q) que será usada
    - num_episodes: quantidade de episódios a serem executados
    - render: defina como True se deseja chamar `env.render()` a cada passo
    - render_wait: intervalo de tempo entre as chamadas a `env.render()`
    - video 
    
    Retorna:
    - um par contendo o valor escalar do retorno médio por episódio e 
       a lista de retornos de todos os episódios
    """
    episode_returns = []
    total_steps = 0
    for i in range(num_episodes):
        print(f"Episode {i+1}")
        obs = env.reset()
        if render:
            env.render()
            time.sleep(render_wait)
        if video is not None:
            video.capture_frame()
        done = False
        episode_returns.append(0.0)
        while not done:
            action = np.argmax(Q[obs])
            obs, reward, done, _ = env.step(action)
            if render:
                env.render()
                time.sleep(render_wait)
            if video is not None:
                video.capture_frame()
            total_steps += 1
            episode_returns[-1] += reward
        print("- retorno:", episode_returns[-1])
    mean_return = round(np.mean(episode_returns), 1)
    print("Retorno médio (por episódio):", mean_return, end="")
    print(", episódios:", len(episode_returns), end="")
    print(", total de passos:", total_steps)
    show_state(env,total_steps)
    if video is not None:
        video.close()
    return mean_return, episode_returns

In [ ]:
def smooth(data, window):
  data = np.array(data)
  n = len(data)
  y = np.zeros(n)
  for i in range(n):
    start = max(0, i-window+1)
    y[i] = data[start:(i+1)].mean()
  return y

def plot_result(returns, ymax_suggested=None, window=100, filename=None):
    '''Exibe um gráfico "retornos x recompensas", fazendo a média a cada 100 retornos, para suavizar.     
    Se o parâmetro filename for fornecido, salva o gráfico em arquivo ao invés de exibir.
    
    Parâmetros:
    - returns: lista de retornos a cada episódio
    - ymax_suggested (opcional): valor máximo de retorno (eixo y), se tiver um valor máximo conhecido previamente
    - filename: indique um nome de arquivo, se quiser salvar a imagem do gráfico; senão, o gráfico será apenas exibido
    '''
    plt.figure(figsize=(14,8))
    smoothed_returns = smooth(returns, window)
    xvalues = np.arange(1, len(returns)+1)
    plt.plot(xvalues, smoothed_returns)
    plt.xlabel('Episódios')
    plt.ylabel('Retorno')
    if ymax_suggested is not None:
        ymax = np.max([ymax_suggested, np.max(smoothed_returns)])
        plt.ylim(top=ymax)
    plt.title(f"Retorno médio a cada {window} episódios")
    if filename is None:
        plt.show()
    else:
        plt.savefig(filename)
        print("Arquivo salvo:", filename)
    plt.close()


# Exibe o ambiente

In [ ]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder

In [ ]:
ENV_NAME = 'MountainCar-v0'

In [ ]:
env = gym.make(ENV_NAME)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [ ]:
def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

In [ ]:
#record_video(ENV_NAME, model, video_length=1000, prefix='monte-car-off-police')

videoOf = VideoRecorder(env, "monte-car-off-police.mp4")
#videoOn = VideoRecorder(env, "monte-car-on-police.mp4")
#test_greedy_Q_policy(env, QtableOn, 10, True,videoOn)
#render_mp4('video_offpolicy')

test_greedy_Q_policy(env, QtableOn, 10, True,videoOf)
render_mp4('monte-car-off-police.mp4')
#show_videos('videos', prefix='monte-car-off-police')